In [5]:
import pandas as pd
import numpy as np

from ast import literal_eval                                  # 문자열 파싱 라이브러리
from sklearn.feature_extraction.text import CountVectorizer   # 
from sklearn.metrics.pairwise import cosine_similarity        # 코사인 유사도 라이브러리

import warnings

warnings.filterwarnings('ignore')

# 걍 한 번에 처리하기

In [3]:
def find_sim_beer_ver1(beer_data, sorted_ind, beer_name, top_n=3):
    
    beer_name = beer_data[beer_data['맥주이름'] == beer_name]
    
    beer_index = beer_name.index.values
    similar_indexes = sorted_ind[beer_index, :(top_n)]
    
    similar_inidexes = similar_indexes.reshape(-1)
    
    return beer_data.iloc[similar_inidexes]

In [58]:
# 데이터 가져오기(맥주 데이터)
beer_load = pd.read_csv('맥주이름_특징_tag_완료_최종.csv',encoding='utf-8', index_col=0)
rating = pd.read_csv('맥주_10.csv', encoding='utf-8', index_col=0)

In [59]:
# beer_rating: 맥주 종류별 평점평균
beer_rating = pd.DataFrame(columns=['맥주이름','평점평균','평가횟수'])
beer_rating['맥주이름'] = list(set(rating['맥주']))
beer_rating

# 평점평균 구하기
for beer in beer_rating['맥주이름']:
    temp = rating[beer==rating['맥주']]['평점'].mean()
    beer_rating['평점평균'][beer_rating['맥주이름']==beer]=temp
    
# 평가횟수 구하기
for beer in beer_rating['맥주이름']:
    temp = len(rating[beer==rating['맥주']])
    beer_rating['평가횟수'][beer_rating['맥주이름']==beer]=temp
    
# 총 데이터 합병
beer_data = pd.merge(beer_load, beer_rating)
    
# 데이터 준비 및 정재 list 형태로 바꿔주기 
# beer_data에서 총 6개의 feature(Aroma, Flavor, Balance, Season, Paring  Food, Body)를 가져온다 
beer_df = pd.DataFrame(beer_data, columns = ['Aroma','Flavor','Balance','Season','Paring Food', 'Body'])

# str형태에서 list로 변환
beer_df['Aroma'] = beer_df['Aroma'].apply(literal_eval)
beer_df['Flavor'] = beer_df['Flavor'].apply(literal_eval)
beer_df['Balance'] = beer_df['Balance'].apply(literal_eval)
beer_df['Season'] = beer_df['Season'].apply(literal_eval)
beer_df['Paring Food'] = beer_df['Paring Food'].apply(literal_eval)
beer_df['Body'] = beer_df['Body'].apply(literal_eval)

In [9]:
# ========================================= #
# 데이터 로드
beer_data = pd.read_csv('맥주_cbf_data.csv', encoding='utf-8')
beer_df = pd.read_csv('맥주_cbf_feature.csv', encoding='utf-8')

# str형태에서 list로 변환
beer_df['Aroma'] = beer_df['Aroma'].apply(literal_eval)
beer_df['Flavor'] = beer_df['Flavor'].apply(literal_eval)
beer_df['Balance'] = beer_df['Balance'].apply(literal_eval)
beer_df['Season'] = beer_df['Season'].apply(literal_eval)
beer_df['Paring Food'] = beer_df['Paring Food'].apply(literal_eval)
beer_df['Body'] = beer_df['Body'].apply(literal_eval)

# something = 'Aroma'
# something = 'Flavor'
# something = 'Balance'
# something = 'Paring Food'
something = 'Body'

# CountVectorizer를 활용하기 위한 전처리 
beer_df[something +'_literal'] = beer_df[something].apply(lambda x : (' ').join(x))

#CountVectorizer로 학습시키기
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))   # 특징 2개(ngram_range: 1<= n <=2)
something_mat = count_vect.fit_transform(beer_df[something + '_literal'])

# 77개의 맥주에 대한 가각 유사한 맥주들이 계산 됨
something_sim = cosine_similarity(something_mat,something_mat)

# 유사도 높은순으로 역순 배치 후 인덱스 추출
something_sim_sorted_ind = something_sim.argsort()[:, ::-1]

# 10개의 맥주 추천 결과
# Pilsner Urquel로 검색한 결과, 대체로 필스터 혹은 비슷한 성질을 가진 페일에일이 추천되는 것을 알 수 있다.

beer_name = 'Cass Fresh'

similar_beers = find_sim_beer_ver1(beer_data, something_sim_sorted_ind, beer_name, 4)

similar_beers = similar_beers[similar_beers['맥주이름'] != beer_name]

similar_beers

,Unnamed: 0,Unnamed: 0.1,맥주이름,맥주스타일,Main Category,Aroma,Flavor,Balance,Season,Paring Food,Body,평점평균,평가횟수
5,5,7,Stella Artois,Pale Lager - International / Premium,라거,"['고소한 맥아향', '강한 향', '매운 향', '플로랄 홉향', '가벼운 향']","['쓴 맛', '상쾌함', '약간의 단맛', '맥아 풍미']",['미디엄'],['연중 내내'],"['생선 요리', '해산물', '샐러드']","['드라이', '깔끔함', '가벼움', '적당한 탄산']",2.452250,1822
27,27,31,Budweiser,Pale Lager - American,라거,"['고소한 맥아향', '강한 향']","['쓴 맛', '상쾌함']",['미디엄'],['연중 내내'],"['생선 요리', '해산물', '샐러드']","['드라이', '깔끔함', '가벼움', '적당한 탄산']",1.486199,1739
0,0,0,Kloud Original Gravity,Pale Lager - International / Premium,라거,"['고소한 맥아향', '강한 향', '빵 향', '달콤한 향', '풀잎 향']","['쓴 맛', '상쾌함', '생강 힌트']",['미디엄'],['연중 내내'],"['생선 요리', '해산물', '샐러드']","['드라이', '깔끔함', '가벼움', '탄산']",2.292500,80


In [12]:
tmp = beer_data[beer_data['맥주이름']==beer_name]
tmp

,Unnamed: 0,Unnamed: 0.1,맥주이름,맥주스타일,Main Category,Aroma,Flavor,Balance,Season,Paring Food,Body,평점평균,평가횟수
7,7,13,Cass Fresh,Pale Lager - International / Premium,라거,"['고소한 맥아향', '강한 향', '가벼운 향']","['쓴 맛', '상쾌함', '약간 단맛', '김치에 좋은 맥주']",['미디엄'],['연중 내내'],"['생선 요리', '해산물', '샐러드']","['드라이', '깔끔함', '가벼움', '적당한 탄산']",1.74823,226


In [ ]:
result = ['Cass Fresh', 'A', 'B']

In [ ]:
category=[], food=[], select=[]
for i in range(len(result)):
    category.append(beer_data[beer_data['맥주이름']==result[i]])
    select.

In [25]:
beer_data[beer_data['맥주이름']=='Cass Fresh']['Paring Food'].values[0]

"['생선 요리', '해산물', '샐러드']"

In [27]:
beer_data.맥주이름.unique()

array(['Kloud Original Gravity', 'Asahi Super Dry', 'Tsingtao',
       'Heineken', 'Kirin Ichiban', 'Stella Artois', 'San Miguel',
       'Cass Fresh', 'Victoria Bitter', 'Bintang Pilsener',
       "Suntory The Premium Malt's", 'Tiger Beer', 'Sapporo Yebisu',
       'Peroni Nastro Azzurro', 'Holsten Premium Bier',
       'Lapin Kulta IV A', 'Grolsch Premium Lager',
       'Castlemaine XXXX Gold Lager', 'Löwenbräu Original (4.2%)',
       'Harbin Beer', "Beck's", 'Kloud Fitz Super Clear',
       'OB Premier Pilsner', 'Hite D (Dry Finish)', 'Hite Prime Max',
       'Hite Extra Cold', 'Miller Genuine Draft (MGD)', 'Budweiser',
       'Stephans Bräu Premium Lager', 'Guinness Draught',
       'Kronenbourg 1664 Blanc', 'Jeju Wit Ale', 'Pilsner Urquell',
       'Gambrinus Originál 10° (Desítka)', 'Krombacher Weizen',
       'Tsingtao Wheat Beer', 'Erdinger Weissbier',
       'Paulaner Hefe-Weissbier', 'Edelweiss Weissbier Snowfresh',
       'Stephans Bräu Hefe-Weizen Naturtrüb',
       'Volfa

In [33]:
beer_data[beer_data['맥주이름']=='Edelweiss Weissbier Snowfresh']['Main Category'].values

array(['컨티넨털 에일/바이스비어/에일-라거하이브리드'], dtype=object)

In [34]:
beer_data[beer_data['맥주이름']=='Edelweiss Weissbier Snowfresh']['Paring Food'].values

array(["['샐러드', '해산물', '가벼운 음식', '바이스부르스트', '바나나 케이크', '바베큐 케이준', '저크 피시', '닭고기', '돼지고기', '애플 파이', '매운 음식', '태국 음식', '커리', '오믈렛', '에그 베네딕트', '베이컨', '달걀']"],
      dtype=object)

In [52]:
category = similar_beers['Main Category'].tolist()

In [55]:
select = similar_beers['Flavor'].tolist()
select

["['달콤한 맥아', '캐러멜', '사츠 홉의 풍미', '쌉쌀한 풍미', '쓴 맛', '섬세함']",
 "['달콤한 맥아', '캐러멜', '사츠 홉의 풍미', '쌉쌀한 풍미', '쓴 맛', '섬세함', '단 맛', '빵 껍질']",
 "['강한 쓴맛', '쓴 맛', '섬세함', '캐러멜', '단맛']"]

In [51]:
aroma = similar_beers['Aroma'].tolist()
aroma[2]

"['깔끔한 맥아향', '허브향', '민트향', '노블 홉향', '강한 향', '소나무향', '약한 빵 향']"

In [44]:
similar_beers['Main Category'].tolist()

['라거', '라거', '라거']

In [21]:
result = similar_beers['맥주이름'].tolist()

In [35]:
result[0]

'Gambrinus Originál 10° (Desítka)'

In [40]:
# 총 데이터 합병
beer_data = pd.merge(beer_load, beer_rating)
beer_data.tail(50)

,Unnamed: 0.1,맥주이름,맥주스타일,Main Category,Aroma,Flavor,Balance,Season,Paring Food,Body,평점평균,평가횟수
10,24,Suntory The Premium Malt's,Pale Lager - International / Premium,라거,"['고소한 맥아향', '강한 향', '거친 맥아향']","['쓴 맛', '상쾌함']",['미디엄'],['연중 내내'],"['생선 요리', '해산물', '샐러드']","['드라이', '깔끔함', '적당한 탄산']",2.65205,146
11,27,Tiger Beer,Pale Lager - International / Premium,라거,"['고소한 맥아향', '강한 향', '달콤한 맥아향']","['쓴 맛', '상쾌함', '옥수수 단맛']","['미디엄', '홉에 의한 밸런스 거의 없음']",['연중 내내'],"['생선 요리', '해산물', '샐러드', '아시아 음식']","['드라이', '깔끔함', '적당한 탄산']",2.11764,1253
12,33,Sapporo Yebisu,Pale Lager - International / Premium,라거,"['고소한 맥아향', '강한 향', '달콤한 맥아향']","['쓴 맛', '상쾌함', '약한 단맛']",['미디엄'],['연중 내내'],"['생선 요리', '해산물', '샐러드']","['드라이', '깔끔함', '적당한 탄산화']",2.47789,285
13,36,Peroni Nastro Azzurro,Pale Lager - International / Premium,라거,"['고소한 맥아향', '강한 향', '스컹크 향', '풀잎 향']","['쓴 맛', '상쾌함']",['미디엄'],['연중 내내'],"['생선 요리', '해산물', '샐러드']","['드라이', '깔끔함', '가벼움']",2.16743,1182
14,51,Holsten Premium Bier,Pale Lager - International / Premium,라거,"['고소한 맥아향', '강한 향']","['쓴 맛', '상쾌함', '단 맛']","['미디엄', '크리미']",['연중 내내'],"['생선 요리', '해산물', '샐러드']","['드라이', '깔끔함']",2.33961,361
15,56,Lapin Kulta IV A,Pale Lager - International / Premium,라거,"['고소한 맥아향', '강한 향', '약한 향', '약한 스컹스 향', '약한 과일향']","['쓴 맛', '상쾌함']",['미디엄'],['연중 내내'],"['생선 요리', '해산물', '샐러드']","['드라이', '깔끔함', '가벼움']",2.20375,320
16,57,Grolsch Premium Lager,Pale Lager - International / Premium,라거,"['고소한 맥아향', '강한 향', '풀잎 향', '쿠키향', '약한 스컹크 향']","['쓴 맛', '상쾌함']","['미디엄', '크리미']",['연중 내내'],"['생선 요리', '해산물', '샐러드']","['드라이', '깔끔함', '적당한 탄산화']",2.51545,1469
17,59,Castlemaine XXXX Gold Lager,Pale Lager - International / Premium,라거,"['고소한 맥아향', '강한 향', '과일향']","['쓴 맛', '상쾌함']",['미디엄'],['연중 내내'],"['생선 요리', '해산물', '샐러드']","['드라이', '깔끔함']",1.83333,129
18,61,Löwenbräu Original (4.2%),Pale Lager - International / Premium,라거,"['고소한 맥아향', '강한 향', '달콤한 맥아향', '꽃 향기']","['쓴 맛', '상쾌함', '단 맛', '옥수수 맛']","['미디엄', '크리미']","['연중 내내', '봄']","['생선 요리', '해산물', '샐러드']","['드라이', '깔끔함', '가벼움']",1.95909,22
19,63,Harbin Beer,Pale Lager - International / Premium,라거,"['고소한 맥아향', '강한 향', '옥수수 향']","['쓴 맛', '상쾌함', '단 맛']",['미디엄'],['연중 내내'],"['생선 요리', '해산물', '샐러드']","['드라이', '깔끔함', '많은 탄산']",2.02442,258


In [41]:
beer_load.tail(50)

,Unnamed: 0.1,맥주이름,맥주스타일,Main Category,Aroma,Flavor,Balance,Season,Paring Food,Body
27,16,Hite Prime Max,Pale Lager - American,라거,"['고소한 맥아향', '강한 향', '풀잎향']","['쓴 맛', '상쾌함']",['미디엄'],['연중 내내'],"['생선 요리', '해산물', '샐러드']","['드라이', '깔끔함']"
28,17,Hite Extra Cold,Pale Lager - American,라거,"['고소한 맥아향', '강한 향']","['쓴 맛', '상쾌함', '옅은 옥수수 향', '옅은 맥아향']","['미디엄', '단 맛']",['연중 내내'],"['생선 요리', '해산물', '샐러드']","['드라이', '깔끔함']"
29,21,Miller Genuine Draft (MGD),Pale Lager - American,라거,"['고소한 맥아향', '강한 향', '옥수수 향', '약한 캐라멜 멕아향']","['쓴 맛', '상쾌함', '캐러멜']",['미디엄'],['연중 내내'],"['생선 요리', '해산물', '샐러드']","['드라이', '깔끔함']"
30,31,Budweiser,Pale Lager - American,라거,"['고소한 맥아향', '강한 향']","['쓴 맛', '상쾌함']",['미디엄'],['연중 내내'],"['생선 요리', '해산물', '샐러드']","['드라이', '깔끔함', '가벼움', '적당한 탄산']"
31,41,FiLite,Pale Lager - American,라거,"['고소한 맥아향', '강한 향']","['쓴 맛', '상쾌함', '역겨움']",['미디엄'],['연중 내내'],"['생선 요리', '해산물', '샐러드']","['드라이', '깔끔함']"
32,45,Stephans Bräu Premium Lager,Pale Lager - American,라거,"['고소한 맥아향', '강한 향', '꽃향기', '달콤한 맥아향']","['쓴 맛', '상쾌함', '약한 단맛']",['미디엄'],['연중 내내'],"['생선 요리', '해산물', '샐러드']","['드라이', '깔끔함', '가벼움', '풍부한 탄산']"
33,8,Guinness Draught,Stout - Dry / Irish,영국-아일랜드 에일,"['로스팅 보리향', '커피향', '홉 아로마 거의 없음', '약한 빵 향']","['쓴 맛', '약간 크리미', '초콜렛']","['매우 드라이', '청량함']",['연중 내내'],"['스테이크', '고기 파이', '리치한 음식', '든든한 음식', '굴', '초콜릿']","['드라이', '가벼운 탄산화', '부드러움']"
34,9,Kronenbourg 1664 Blanc,Witbier / Belgian White Ale,컨티넨털 에일/바이스비어/에일-라거하이브리드,"['스파이시한 효모향', '오렌지', '고수', '과일향']","['드라이한 크림 풍미', '부드러움', '시큼한 끝맛', '사과맛']","['밀크셰이크 질감', '드라이', '약간 시큼함', '크리미']","['연중 내내', '따뜻한 날씨']","['홍합', '연어', '닭', '가벼운 음식']","['드라이', '미디엄', '매우 크리미', '크리미']"
35,43,Jeju Wit Ale,Witbier / Belgian White Ale,컨티넨털 에일/바이스비어/에일-라거하이브리드,"['스파이시한 효모향', '오렌지', '고수', '순한 고수향', '밀 향']","['드라이한 크림 풍미', '부드러움', '시큼한 끝맛']","['밀크셰이크 질감', '드라이', '약간 시큼함']","['연중 내내', '따뜻한 날씨']","['홍합', '연어', '닭', '가벼운 음식']","['드라이', '미디엄', '매우 크리미', '크리미']"
36,66,Platinum White Ale,Witbier / Belgian White Ale,컨티넨털 에일/바이스비어/에일-라거하이브리드,"['스파이시한 효모향', '오렌지', '고수']","['드라이한 크림 풍미', '부드러움', '시큼한 끝맛']","['밀크셰이크 질감', '드라이', '약간 시큼함']","['연중 내내', '따뜻한 날씨']","['홍합', '연어', '닭', '가벼운 음식']","['드라이', '미디엄', '매우 크리미', '크리미']"


In [42]:
# 데이터 준비 및 정재 list 형태로 바꿔주기 
# beer_data에서 총 6개의 feature(Aroma, Flavor, Balance, Season, Paring  Food, Body)를 가져온다 
beer_df = pd.DataFrame(beer_data, columns = ['Aroma','Flavor','Balance','Season','Paring Food', 'Body'])


# 데이터 타입 확인: data셀 하나에 들어있는 내용들은 str
type(beer_df['Aroma'].iloc[0])
beer_df['Aroma'].iloc[0]


# str형태에서 list로 변환
beer_df['Aroma'] = beer_df['Aroma'].apply(literal_eval)
beer_df['Flavor'] = beer_df['Flavor'].apply(literal_eval)
beer_df['Balance'] = beer_df['Balance'].apply(literal_eval)
beer_df['Season'] = beer_df['Season'].apply(literal_eval)
beer_df['Paring Food'] = beer_df['Paring Food'].apply(literal_eval)
beer_df['Body'] = beer_df['Body'].apply(literal_eval)

In [43]:
beer_df

,Aroma,Flavor,Balance,Season,Paring Food,Body
0,"[고소한 맥아향, 강한 향, 빵 향, 달콤한 향, 풀잎 향]","[쓴 맛, 상쾌함, 생강 힌트]",[미디엄],[연중 내내],"[생선 요리, 해산물, 샐러드]","[드라이, 깔끔함, 가벼움, 탄산]"
1,"[고소한 맥아향, 강한 향, 쌀 향, 풀잎 향]","[쓴 맛, 상쾌함, 쌀 풍미, 약간의 단맛, 약간의 쓴 맛]",[미디엄],[연중 내내],"[생선 요리, 해산물, 샐러드]","[드라이, 깔끔함, 가벼움, 약간의 매끄러움]"
2,"[고소한 맥아향, 강한 향, 희미한 옥수수향]","[쓴 맛, 상쾌함, 가벼운 맥아 풍미, 옥수수 풍미, 약간의 쓴맛]",[미디엄],[연중 내내],"[생선 요리, 해산물, 샐러드, 매운 음식]","[드라이, 깔끔함, 가벼움, 가벼운 탄산]"
3,"[고소한 맥아향, 강한 향, 가벼운 스컹스 향, 풀잎 향]","[쓴 맛, 상쾌함, 스컹크 풍미]",[미디엄],[연중 내내],"[생선 요리, 해산물, 샐러드]","[드라이, 깔끔함, 탄산]"
4,"[고소한 맥아향, 강한 향, 가벼운 낱알 몰트 향]","[쓴 맛, 상쾌함, 맥아 풍미]",[미디엄],[연중 내내],"[생선 요리, 해산물, 샐러드]","[드라이, 깔끔함, 적당한 탄산]"
5,"[고소한 맥아향, 강한 향, 매운 향, 플로랄 홉향, 가벼운 향]","[쓴 맛, 상쾌함, 약간의 단맛, 맥아 풍미]",[미디엄],[연중 내내],"[생선 요리, 해산물, 샐러드]","[드라이, 깔끔함, 가벼움, 적당한 탄산]"
6,"[고소한 맥아향, 강한 향, 약한 향, 과일향]","[쓴 맛, 상쾌함, 보조 맥아 힌트, 사과 주스 맛]",[미디엄],"[연중 내내, 더운 여름]","[생선 요리, 해산물, 샐러드]","[드라이, 깔끔함, 적당한 탄산]"
7,"[고소한 맥아향, 강한 향, 가벼운 향]","[쓴 맛, 상쾌함, 약간 단맛, 김치에 좋은 맥주]",[미디엄],[연중 내내],"[생선 요리, 해산물, 샐러드]","[드라이, 깔끔함, 가벼움, 적당한 탄산]"
8,"[고소한 맥아향, 강한 향, 스파이시한 향, 약한 빵 향]","[쓴 맛, 상쾌함, 순함, 약한 쓴맛]","[미디엄, 균형]",[연중 내내],"[생선 요리, 해산물, 샐러드]","[드라이, 깔끔함, 부드러움, 가벼움, 탄산]"
9,"[고소한 맥아향, 강한 향, 가벼운 향]","[쓴 맛, 상쾌함, 약한 달콤한 맥아]",[미디엄],[연중 내내],"[생선 요리, 해산물, 샐러드]","[드라이, 깔끔함, 가벼움]"


In [65]:
beer_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60 entries, 0 to 59
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Aroma         60 non-null     object
 1   Flavor        60 non-null     object
 2   Balance       60 non-null     object
 3   Season        60 non-null     object
 4   Paring Food   60 non-null     object
 5   Body          60 non-null     object
 6   Body_literal  60 non-null     object
dtypes: object(7)
memory usage: 3.8+ KB


# 추출한 Feature별 컨텐츠 유사도 측정

## Something에 무엇을 넣느냐에 따라 달라진다.
### 총 5개의 항목 'Aroma', 'Flavor', 'Balance', 'Paring Food', 'Body'

In [44]:
# something = 'Aroma'
# something = 'Flavor'
# something = 'Balance'
# something = 'Paring Food'
something = 'Body'

In [45]:
# CountVectorizer를 활용하기 위한 전처리 
beer_df[something +'_literal'] = beer_df[something].apply(lambda x : (' ').join(x))
beer_df[something +'_literal']

0              드라이 깔끔함 가벼움 탄산
1        드라이 깔끔함 가벼움 약간의 매끄러움
2          드라이 깔끔함 가벼움 가벼운 탄산
3                  드라이 깔끔함 탄산
4              드라이 깔끔함 적당한 탄산
5          드라이 깔끔함 가벼움 적당한 탄산
6              드라이 깔끔함 적당한 탄산
7          드라이 깔끔함 가벼움 적당한 탄산
8         드라이 깔끔함 부드러움 가벼움 탄산
9                 드라이 깔끔함 가벼움
10             드라이 깔끔함 적당한 탄산
11             드라이 깔끔함 적당한 탄산
12            드라이 깔끔함 적당한 탄산화
13                드라이 깔끔함 가벼움
14                    드라이 깔끔함
15                드라이 깔끔함 가벼움
16            드라이 깔끔함 적당한 탄산화
17                    드라이 깔끔함
18                드라이 깔끔함 가벼움
19              드라이 깔끔함 많은 탄산
20          드라이 깔끔함 가벼움 적은 탄산
21                    드라이 깔끔함
22                 드라이 깔끔함 탄산
23                    드라이 깔끔함
24                    드라이 깔끔함
25                    드라이 깔끔함
26                    드라이 깔끔함
27         드라이 깔끔함 가벼움 적당한 탄산
28         드라이 깔끔함 가벼움 풍부한 탄산
29           드라이 가벼운 탄산화 부드러움
30         드라이 미디엄 매우 크리미 크리미
31         드라이 미디엄 매우 크리미 크리미
32             미디엄 깔끔함 탄산 가벼움
33        

In [46]:
#CountVectorizer로 학습시키기
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))   # 특징 2개(ngram_range: 1<= n <=2)
something_mat = count_vect.fit_transform(beer_df[something + '_literal'])

# 77개의 맥주 중에 178개 Aroma의 'Aroma 매트릭스'가 생성됨
print(something_mat.shape)

(60, 65)


In [48]:
something_mat

<60x65 sparse matrix of type '<class 'numpy.int64'>'
	with 329 stored elements in Compressed Sparse Row format>

In [47]:
# 77개의 맥주 중에 65개 Aroma의 'Aroma 매트릭스'가 생성됨
count_vect2 = CountVectorizer(min_df=1, ngram_range=(1,1))   
something_mat2 = count_vect2.fit_transform(beer_df[something+'_literal'])
print(something_mat2.shape)

(60, 23)


In [49]:
something_mat2

<60x23 sparse matrix of type '<class 'numpy.int64'>'
	with 193 stored elements in Compressed Sparse Row format>

In [22]:
# 77개의 맥주에 대한 가각 유사한 맥주들이 계산 됨

something_sim = cosine_similarity(something_mat,something_mat)
print(something_sim.shape)
print(something_sim)

(60, 60)
[[1.         0.62994079 0.75592895 ... 0.         0.37796447 0.21821789]
 [0.62994079 1.         0.55555556 ... 0.         0.33333333 0.19245009]
 [0.75592895 0.55555556 1.         ... 0.         0.33333333 0.19245009]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.37796447 0.33333333 0.33333333 ... 0.         1.         0.        ]
 [0.21821789 0.19245009 0.19245009 ... 0.         0.         1.        ]]


In [50]:
# 순서 보기
# 숫자가 작을수록 유사도가 높은 맥주
something_sim_sorted_ind = something_sim.argsort()[:, ::-1]
print(something_sim_sorted_ind[:1])

[[ 0 18  9 13 15  2  5 28 20 27  8  7  3 22 24 23 26 17 21 14 25  1 19 11
   4  6 10 56 41 12 47 49 48 45 32 33 16 43 58 46 34 39 50 36 59 37 38 40
  44 52 53 54 42 35 29 30 31 51 55 57]]


# 코사인 유사도로 맥주 추천

In [51]:
def find_sim_beer_ver1(beer_data, sorted_ind, beer_name, top_n=10):
    
    beer_name = beer_data[beer_data['맥주이름'] == beer_name]
    
    beer_index = beer_name.index.values
    similar_indexes = sorted_ind[beer_index, :(top_n)]
    
    print(similar_indexes)
    
    similar_inidexes = similar_indexes.reshape(-1)
    
    return beer_data.iloc[similar_inidexes]

In [52]:
# 10개의 맥주 추천 결과
# Pilsner Urquel로 검색한 결과, 대체로 필스터 혹은 비슷한 성질으 가진 페일에일이 추천되는 것을 알 수 있다.

similar_beers = find_sim_beer_ver1(beer_data, something_sim_sorted_ind, 'Pilsner Urquell',10)
similar_beers[['맥주이름','맥주스타일',something,'평점평균','평가횟수']]

[[33 32 56 41 22  3 53 52  0 28]]


,맥주이름,맥주스타일,Body,평점평균,평가횟수
33,Gambrinus Originál 10° (Desítka),Pilsener - Bohemian / Czech,"['미디엄', '깔끔함', '탄산', '가벼움']",2.51777,332
32,Pilsner Urquell,Pilsener - Bohemian / Czech,"['미디엄', '깔끔함', '탄산', '가벼움']",3.24791,2077
56,Bavaria 8.6 (Original),Pilsener - Imperial,"['깔끔함', '탄산', '가벼움']",2.17543,293
41,Tiger Radler Lemon,Radler / Shandy,"['탄산', '가벼움', '깔끔함']",2.01,20
22,OB Premier Pilsner,Pale Lager - American,"['드라이', '깔끔함', '탄산']",2.18571,49
3,Heineken,Pale Lager - International / Premium,"['드라이', '깔끔함', '탄산']",2.08413,1953
53,Asahi Super Dry Black,Dark Lager - International / Premium,"['미디엄', '가벼움']",2.62525,99
52,Heineken Dark Lager,Dark Lager - International / Premium,"['미디엄', '가벼움']",2.65792,404
0,Kloud Original Gravity,Pale Lager - International / Premium,"['드라이', '깔끔함', '가벼움', '탄산']",2.2925,80
28,Stephans Bräu Premium Lager,Pale Lager - American,"['드라이', '깔끔함', '가벼움', '풍부한 탄산']",2.04615,39


In [53]:
beer_data[['맥주이름','맥주스타일',something,'평점평균','평가횟수']].sort_values('평점평균', ascending=False)[:3]

,맥주이름,맥주스타일,Body,평점평균,평가횟수
37,Paulaner Hefe-Weissbier,Weissbier - Hefeweizen,"['걸쭉', '드라이']",3.55755,1807
50,Hoegaarden,Wheat Ale,"['걸쭉', '드라이', '가벼움']",3.46274,2048
29,Guinness Draught,Stout - Dry / Irish,"['드라이', '가벼운 탄산화', '부드러움']",3.3069,2100


## 가중평점(평점&평가횟수) 반영한 영화 추천

In [54]:
# 실습에서 한 식 그대로 사용
# 상위 60%에 해당하는 평가 횟수를 최소 투표 횟수인 m으로 활용
C = beer_data['평점평균'].mean()
m = beer_data['평가횟수'].quantile(0.6)

In [55]:
print('C:',round(C,3), 'm:',round(m,3))

C: 2.396 m: 500.8


## 가중평점을 계산하는 함수

In [56]:
# 가중평점을 계산하는 함수
def weighted_vote_average(data):
    v = data['평가횟수']
    R = data['평점평균']
    
    return ( (v/(v+m)) * R) + (m/(m+v) * C)

In [57]:
# 기존 데이터에 가중평점 칼럼 추가
beer_data['가중평점'] = beer_data.apply(weighted_vote_average, axis=1)

In [58]:
# 추천을 받기위해 넣었던 데이터
beer_data[beer_data['맥주이름']=='Pilsner Urquell'][['Main Category','맥주스타일','맥주이름',something,
           '평점평균','평가횟수']]

,Main Category,맥주스타일,맥주이름,Body,평점평균,평가횟수
32,라거,Pilsener - Bohemian / Czech,Pilsner Urquell,"['미디엄', '깔끔함', '탄산', '가벼움']",3.24791,2077


In [60]:
# 가중 평균까지 구하여 넣은 값
beer_data[['가중평점','맥주이름','맥주스타일','Main Category', something,
           '평점평균','평가횟수']].sort_values('가중평점', ascending=False)[:3]

,가중평점,맥주이름,맥주스타일,Main Category,Body,평점평균,평가횟수
37,3.305564,Paulaner Hefe-Weissbier,Weissbier - Hefeweizen,컨티넨털 에일/바이스비어/에일-라거하이브리드,"['걸쭉', '드라이']",3.55755,1807
50,3.253210,Hoegaarden,Wheat Ale,벨기에 맥주,"['걸쭉', '드라이', '가벼움']",3.46274,2048
29,3.131567,Guinness Draught,Stout - Dry / Irish,영국-아일랜드 에일,"['드라이', '가벼운 탄산화', '부드러움']",3.3069,2100


### 추천 ver2. 먼저 장르 유사성 높은 영화 20개 선정 후 (F1), 가중 평점순 10개 선정

#### 하나의 맥주에 대해 장르 유사성, 가중평점 반영한 추천 맥주 10개를 뽑아보자

In [61]:
def find_sim_beer_ver2(data, sorted_ind, beer_name, top_n=10):
    beer_name = data[data['맥주이름'] == beer_name]
    beer_index = beer_name.index.values
    
    # top_n의 2배에 해당하는 쟝르 유사성이 높은 index 추출
    similar_indexes = sorted_ind[beer_index, :(top_n*2)]
    similar_indexes = similar_indexes.reshape(-1)

    # 기준 영화 index는 제외(자기자신 제외)
    similar_indexes = similar_indexes[similar_indexes != beer_index]
    
    # top_n의 2배에 해당하는 후보군에서 weighted_vote 높은 순으로 top_n 만큼 추출 
    return data.iloc[similar_indexes].sort_values('가중평점', ascending=False)[:top_n]

In [63]:
similar_beers = find_sim_beer_ver2(beer_data, something_sim_sorted_ind, 'Paulaner Hefe-Weissbier',3)
similar_beers[['맥주이름','맥주스타일',something,'평점평균','평가횟수']]

,맥주이름,맥주스타일,Body,평점평균,평가횟수
50,Hoegaarden,Wheat Ale,"['걸쭉', '드라이', '가벼움']",3.46274,2048
59,Erdinger Weissbier Dunkel,Weissbier - Dunkelweizen,"['걸쭉', '드라이']",3.20258,1354
34,Krombacher Weizen,Weissbier - Hefeweizen,"['걸쭉', '드라이', '가벼움']",3.13954,526
